This notebook explores the environmental time series data in relation to extreme events.
The goal is to better understand trends, variability, and basic statistics in python.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dates = pd.date_range("2010-01-01", periods=200)
rainfall = np.random.gamma(shape=2, scale=3, size=200)

dataframe = pd.DataFrame ({
    "date": dates,
    "rainfall_mm": rainfall
})

dataframe.head()